# **Implementation of Doubly Binked List (Base)**

# **Linked Deque**

# **Positional List**

# **Insertion Sort**

# **Favorites List (Using composition pattern, sorting from most rated to least)**

In [ ]:
# Create hidden Doubly Linked List Class

# Then Linked Deque for insert/remove at the front and back

# Then Positional Lit to insert/delete anywhere in the list

# **Doubly Linked List (BASE)**

In [ ]:
class DoublyLinkedBase:

  class Node:

    __slots__="data", "prev", "next"

    def __init__(self, data, prev, next):
      self.data = data
      self.next = next
      self.prev = prev

  def __init__(self):
    self.header = self.Node(None, None, None)
    self.trailer = self.Node(None, None, None)
    self.header.next = self.trailer
    self.trailer.prev = self.header
    self.size = 0

  def __len__(self):
    return self.size

  def is_empty(self):
    return self.size==0

  def insert_between(self, e, predecessor, successor):
    newest = self.Node(e, predecessor, successor)
    predecessor.next = newest
    successor.prev = newest
    self.size +=1
    return newest

  def delete_node(self, node):
    predecessor = node.prev
    successor = node.next
    predecessor.next = successor
    successor.prev = predecessor
    self.size -=1
    data = node.data
    node.prev = node.next = node.data = None
    return data


# **Linked Deque**

In [ ]:
class LinkedDeque(DoublyLinkedBase):

  def first(self):
    if self.is_empty:
      raise Exception("Deque is empty")
    return self.header.next.data

  def last(self):
    if self.is_empty():
      raise Exception("Deque is empty")
    return self.trailer.prev.data

  def insert_first(self, e):
    self.insert_between(e, self.header, self.header.next)

  def insert_last(self,e):
    self.insert_between(e, self.trailer, self.trailer.prev)

  def delete_first(self):
    if self.is_empty():
      raise Exception("Deque is Empty")
    return self.delete_node(self.header.next)

  def delete_last(self):
    if self.is_empty():
      raise Exception("Deque is Empty")
    return self.delete_node(self.trailer.prev)

# **Positional List**

In [ ]:
class PositionalList(DoublyLinkedBase):


  class Position:

    def __init__(self, container, node):
      self.container = container
      self.node = node

    def data(self):
      return self.node.data

    def __eq__(self, other):
      return type(other) is type(self) and other.node is self.node

    def __ne__(self, other):
      return not(self==other)

    # utility method

  def validate(self, p):
    if not isinstance(p, self.Position):
      raise TypeError("p must be proper position type")

    if p.container is not self:
      raise ValueError("P does not belong to this container")

    if p.node.next is None:
      raise ValueError("P is no longer valid")

    return p.node

  def make_position(self, node):
    if node is self.header or node is self.trailer:
      return None
    else:
      return self.Position(self, node)

    # Accessors

  def first(self):
    return self.make_position(self.header.next)

  def last(self):
    return self.make_position(self.trailer.prev)

  def before(self, p):
    node = self.validate(p)
    return self.make_position(node.prev)

  def after(self,p):
    node = self.validate(p)
    return self.make_position(node.next)

  def __iter__(self):
    cursor = self.first()
    while cursor is not None:
      yield cursor.data()
      cursor = self.after(cursor)

    # Mutators :-
  def insert_between(self, e, predecessor, successor):
    node = super().insert_between(e, predecessor, successor)
    return self.make_position(node)

  def add_first(self, e):
    return self.insert_between(e, self.header, self.header.next)

  def add_last(self,e):
    return self.insert_between(e, self.trailer.prev, self.trailer)

  def add_before(self, p, e):
    node = self.validate(p)
    return self.insert_between(e, node.prev, node)

  def add_after(self, p,e):
    node = self.validate(p)
    return self.insert_between(e, node, node.next)

  def delete(self,p):
    node = self.validate(p)
    return super().delete_node(node)

  def replace(self, p, e):
    node = self.validate(p)
    old_value = node.data
    node.data = e
    return old_value

In [ ]:
L = PositionalList()

In [ ]:
L.add_last(36)
L.add_last(22)
L.add_last(15)
L.add_last(29)
L.add_last(23)

In [ ]:
for i in L:
  print(i)

36
22
15
29
23


# **Insertion Sort (Based from above)**

In [ ]:
def insertion_sort(L):

  if len(L)>1:
    marker = L.first()

    while marker != L.last():
      pivot = L.after(marker)
      value = pivot.data()

      if value > marker.data():
        marker = pivot
      else:
        walk = marker

        while walk != L.first() and L.before(walk).data()>value:
          walk = L.before(walk)
        L.delete(pivot)
        L.add_before(walk, value)

In [ ]:
insertion_sort(L)

In [ ]:
print("\nAfter sorting:")
cursor = L.first()
while cursor:
    print(cursor.data(), end=" ")
    cursor = L.after(cursor)


After sorting:
15 22 23 29 36 

# **Using Composition Pattern, Favorites List (Sorted by count)**

In [ ]:
class FavoritesList:

  class item:

    __slots__ = "value", "count"

    def __init__(self, e):
      self.value = e
      self.count = 0

  # Non-public utilities
  def __init__(self):
    self.plist = PositionalList()


  def find_position(self, e):
    walk = self.plist.first()

    while walk is not None and walk.data().value !=e:
      walk = self.plist.after(walk)
    return walk

  def move_up(self,p):
    if p != self.plist.first():
      cnt = p.data().count
      walk = self.plist.before(p)

      if cnt > walk.data().count:
        while (walk != self.plist.first() and cnt > self.plist.before(walk).data().count):
          walk = self.plist.before(walk)
        self.plist.add_before(walk, self.plist.delete(p))

    # Public Methods :-
  def __len__(self):
    return len(self.plist)

  def is_empty(self):
    return len(self.plist)==0

  def access(self, e):
    p = self.find_position(e)

    if p is None:
      p = self.plist.add_last(self.item(e))
    p.data().count += 1
    self.move_up(p)

  def remove(self,e):
    p = self.find_position(e)

    if p is not None:
      self.plist.delete(p)

  def top(self,k):

    if not 1 <= k <= len(self):
      raise ValueError("Illegal value for k")

    walk = self.plist.first()

    for i in range(k):
      item = walk.data()
      yield item.value
      walk = self.plist.after(walk)

In [ ]:
fav = FavoritesList()

In [ ]:
fav.access("Toy Story 3")

In [ ]:
fav.access("Turing")

In [ ]:
fav.access("Toy S")

# **Favourite List Move To The Front**

In [ ]:
classFavoritesListMTF(FavoritesList):
2 ”””Listofelementsorderedwithmove-to-frontheuristic.”””
3
4 #weoverride moveuptoprovidemove-to-frontsemantics
5 def moveup(self,p):
6 ”””MoveaccesseditematPositionptofrontof list.”””
7 ifp!=self. data.first():
8 self. data.addfirst(self. data.delete(p)) #delete/reinsert
9
10 #weoverridetopbecauselist isnolongersorted
11 deftop(self,k):
12 ”””Generatesequenceoftopkelements intermsofaccesscount.”””
13 ifnot1<=k<=len(self):
14 raiseValueError( Illegalvaluefork )
15
16 #webeginbymakingacopyoftheoriginal list
17 temp=PositionalList()
18 for iteminself. data: #positional listssupport iteration
19 temp.add last(item)
20
21 #werepeatedlyfind, report,andremoveelementwithlargestcount
22 for j inrange(k):
23 #findandreportnexthighest fromtemp
24 highPos=temp.first()
25 walk=temp.after(highPos)
26 whilewalkisnotNone:
27 ifwalk.element(). count>highPos.element(). count:
28 highPos=walk
29 walk=temp.after(walk)
30 #wehavefoundtheelementwithhighestcount
31 yieldhighPos.element(). value #reportelementtouser
32 temp.delete(highPos)

SyntaxError: invalid character '”' (U+201D) (ipython-input-1143995314.py, line 2)